# CPLEX IMPLEMENTATION

## 📦 Import packages

In [1]:
import cplex
from cplex import Cplex

import os
import re
import numpy as np
import pandas as pd
import geopandas as gpd

import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

## Prepare Inputs

### Define fixed sets: Time periods and Road widths

In [2]:
def define_fixed_sets():
    """
    Defines and returns fixed sets for time periods and road widths.

    Returns:
    - T: tuple of time periods
    - W: tuple of road widths
    """
    T = tuple([1, 2, 3, 4, 5])  # 5 time periods
    W = tuple(['timber', 'wide'])  # Road widths
    return T, W

### Functions to get component-dependent sets

### ● Load nodes

In [3]:
def load_nodes(component_dir, filename="4withsources_nodes.csv"):
    """
    Load nodes from CSV and return source nodes, exit nodes, and all node coordinates.

    Parameters:
        component_dir (str): Directory containing the nodes CSV.
        filename (str): Name of the nodes CSV file (default: '4withsources_nodes.csv').

    Returns:
        tuple:
            V_source (DataFrame): Subset of nodes_df where is_source == True
            V_exit (DataFrame): Subset of nodes_df where is_exit == True
            V (list): List of (x, y) tuples for all nodes
            nodes_df (DataFrame): Full nodes dataframe
    """
    file_path = f"{component_dir}\\{filename}"
    nodes_df = pd.read_csv(file_path)

    # filter nodes
    V_source = nodes_df[nodes_df["is_source"] == True]
    V_exit = nodes_df[nodes_df["is_exit"] == True]

    # all node coordinates
    V = list(zip(nodes_df["x"], nodes_df["y"]))

    return V_source, V_exit, V, nodes_df


#### Map nodes to ID

In [4]:
def build_node_mappings(V, V_source, V_exit):
    """
    Build node ID mappings and extract node IDs for source and exit nodes.

    Parameters:
        V (list): List of (x, y) coordinates for all nodes.
        V_source (DataFrame): Source nodes (must have 'x' and 'y').
        V_exit (DataFrame): Exit nodes (must have 'x' and 'y').

    Returns:
        tuple:
            node_ID_mapping (dict): Mapping {'nodeX': (x, y)}.
            reversed_node_ID_mapping (dict): Mapping {(x, y): 'nodeX'}.
            V_nodeIDs (list): List of node IDs for all nodes.
            V_source_nIDs (list): List of node IDs for source nodes.
            V_exit_nIDs (list): List of node IDs for exit nodes.
    """
    # forward mapping: node name → coordinates
    node_ID_mapping = {f"node{i + 1}": V[i] for i in range(len(V))}

    # reverse mapping: coordinates → node name
    reversed_node_ID_mapping = {v: k for k, v in node_ID_mapping.items()}

    # all node IDs
    V_nodeIDs = [reversed_node_ID_mapping[v] for v in V]

    # source and exit node IDs
    V_source_nIDs = [reversed_node_ID_mapping[v] for v in zip(V_source["x"], V_source["y"])]
    V_exit_nIDs = [reversed_node_ID_mapping[v] for v in zip(V_exit["x"], V_exit["y"])]

    return node_ID_mapping, reversed_node_ID_mapping, V_nodeIDs, V_source_nIDs, V_exit_nIDs


### Transit nodes

In [5]:
def get_transit_nodes(V_nodeIDs, V_source_nIDs, V_exit_nIDs):
    """
    Compute the set of transit nodes (neither source nor exit).

    Parameters:
        V_nodeIDs (iterable): All node IDs in the graph.
        V_source_dict (dict): Mapping {stand_id: node_id} for source nodes.
        V_exit (iterable): Collection of exit node IDs.

    Returns:
        set: The set of transit node IDs.
    """
    V_transit = set(V_nodeIDs) - set(V_source_nIDs) - set(V_exit_nIDs)

    # Debug: print counts for verification
    print("Total nodes:", len(V_nodeIDs))
    print("Transit nodes:", len(V_transit))
    print("Source nodes:", len(V_source_nIDs))
    print("Exit nodes:", len(V_exit_nIDs))
    return V_transit

### Arcs & Edges

In [6]:
def load_and_verify_arcs(component_dir):
    # Load the arcs from the CSV files
    arcs_fw_df = pd.read_csv(f'{component_dir}/arcsforward.csv', header=0)
    arcs_bw_df = pd.read_csv(f'{component_dir}/arcsbackward.csv', header=0)
    arcs_df = pd.read_csv(f'{component_dir}/arcs.csv', header=0)
    edge_attr_df = pd.read_csv(f'{component_dir}/arcs_with_attributes.csv', header=0)

    # Assign the data to the respective variable
    A = arcs_df.values

    # Print lengths
    print(f"Total arcs: {len(A)}, Forward arcs: {len(arcs_fw_df)}, Backward arcs: {len(arcs_bw_df)}")

    # Check if number of forward arcs equals number of backward arcs
    if len(arcs_fw_df) != len(arcs_bw_df):
        print("Warning: Number of forward arcs is NOT equal to number of backward arcs!")

    # Check if forward arcs are the reverse of backward arcs
    fw_arcs = arcs_fw_df.values
    bw_arcs = arcs_bw_df.values

    if len(fw_arcs) == len(bw_arcs):
        reversed_bw = np.flip(bw_arcs, axis=1)  # Reverse each arc [to, from] -> [from, to]
        if np.array_equal(fw_arcs, reversed_bw):
            print("✅ Forward arcs are the reverse of backward arcs.")
        else:
            print("❌ Forward arcs are NOT the reverse of backward arcs.")
    else:
        print("Cannot check reverse relationship due to different lengths.")

    return A, arcs_fw_df, arcs_bw_df, edge_attr_df


#### use nodes IDs in arcs

In [7]:
# Function to map coordinates to short names
def map_coordinates_to_nodeIDs(coords):
    try:
        coords_tuple = tuple(map(float, coords.strip('()').split(', ')))
    except:
        return None  # handle invalid format
    for nodeID, node_coords in node_ID_mapping.items():
        if coords_tuple == node_coords:
            return nodeID
    return None  # Return None if the coordinates are not found

def replace_coord_with_IDs(A, arcs_fw, arcs_bw):
    # Replace coordinates with their short names
    A = [[map_coordinates_to_nodeIDs(pair[0]), map_coordinates_to_nodeIDs(pair[1])] for pair in A]
    A_fw = [[map_coordinates_to_nodeIDs(pair[0]), map_coordinates_to_nodeIDs(pair[1])] 
            for pair in arcs_fw.values]
    A_bw = [[map_coordinates_to_nodeIDs(pair[0]), map_coordinates_to_nodeIDs(pair[1])] 
            for pair in arcs_bw.values]
    return A, A_fw, A_bw

In [8]:
def process_edge_pairs(A, A_fw, A_bw, verify=True):
    """
    Convert edge pairs into string tuples and verify forward/backward split.

    Parameters:
        A (iterable): Original collection of edge pairs (tuples/lists of chars/strings).
        A_fw (iterable): Forward edge pairs.
        A_bw (iterable): Backward edge pairs.
        verify (bool): If True, checks that A == A_fw ∪ A_bw.

    Returns:
        dict: Dictionary with transformed tuples and verification result.
    """
    # Convert each set of pairs into tuple of concatenated strings
    A = tuple("".join(pair) for pair in A)
    A_fw = tuple("".join(pair) for pair in A_fw)
    A_bw = tuple("".join(pair) for pair in A_bw)
    E = A_fw  # alias for forward edges

    result = {
        "A": A,
        "A_fw": A_fw,
        "A_bw": A_bw,
        "E": E,
        "sample_fw": A_fw[1] if len(A_fw) > 1 else None,
    }

    # Optional verification
    if verify:
        result["verified"] = (set(A) == set(A_fw) | set(A_bw))

    return A, A_fw, A_bw, E

### [helper functions] swap halves and split edge into nodes

In [9]:
# Helper function to swap halves
def swap_direction(arc):
    """
    Swap direction of an arc.
    If arc is a tuple/list (u, v), return (v, u).
    If arc is a string like 'node1node2', split it.
    """
    if isinstance(arc, (tuple, list)) and len(arc) == 2:
        return (arc[1], arc[0])
    elif isinstance(arc, str):
        mid = arc.find('node', 4)  
        return arc[mid:] + arc[:mid] 
    else:
        raise ValueError(f"Unsupported arc format: {arc}")

def split_edge_to_nodes(edge):
    if isinstance(edge, str):
        mid = edge.find('node', 4)  # guaranteed to exist
        return [edge[:mid], edge[mid:]]
    else:
        raise TypeError("Input must be a string.")


In [10]:
def create_forward_backward_mapping(A_fw, A_bw, swap_direction):
    """
    Creates a mapping between forward arcs and backward arcs using a swap function.

    Parameters:
    - A_fw: iterable of forward arcs
    - A_bw: iterable of backward arcs
    - swap_direction: function that takes an arc and returns its swapped version

    Returns:
    - fw_to_bw_map: dict mapping forward arcs to backward arcs
    - bw_to_fw_map: dict mapping backward arcs to forward arcs
    """
    fw_to_bw_map = {}
    for arc in A_fw:
        swapped = swap_direction(arc)
        if swapped in A_bw:
            fw_to_bw_map[arc] = swapped

    bw_to_fw_map = {v: k for k, v in fw_to_bw_map.items()}
    return fw_to_bw_map, bw_to_fw_map

### dict to map StandID: SourceNodeID

In [11]:
def build_source_dict(source_nodes, reversed_node_ID_mapping):
    """
    Build a dictionary mapping stands -> node IDs using source node coordinates.

    Parameters:
        source_nodes (pd.DataFrame): DataFrame with at least 'stands', 'x', 'y' columns.
        reversed_node_ID_mapping (dict): Mapping from (x, y) -> node ID.

    Returns:
        tuple:
            V_source_dict (dict): Mapping {stand_id: node_id}.
            S (set): Set of stand IDs.
    """
    # Initial mapping: stand -> (x, y)
    V_source_dict = {
        int(row['stands']): (row['x'], row['y'])
        for _, row in source_nodes.iterrows()
    }

    # Replace coordinates with node IDs
    for key, node in V_source_dict.items():
        V_source_dict[key] = reversed_node_ID_mapping[node]

    # Define S = stands in V_source
    S = set(V_source_dict.keys())

    return V_source_dict, S

### Source arcs

In [12]:
def get_source_arcs(A, V_source_dict):
    """
    Collect arcs that go into source nodes and out of them.

    Parameters:
        A (iterable): List of arcs, each as a list [u, v] or tuple (u, v).
        V_source_dict (dict): Mapping {stand_id: node_id}.

    Returns:
        dict: {stand_id: set of arcs (ingoing to source)}
        dict: same as above for outgoing
    """
    source_arcs_ingoing = {}
    source_arcs_outgoing = {}

    for standID, sourcenode in V_source_dict.items():
        ingoing = set()
        outgoing = set()
        for arc in A:
            u, v = split_edge_to_nodes(arc)  # unpack list or tuple
            if v == sourcenode:   # incoming arc if target matches source node
                ingoing.add(arc)
            elif u == sourcenode:   # outgoing arc if source matches
                outgoing.add(arc)
        source_arcs_ingoing[standID] = ingoing
        source_arcs_outgoing[standID] = outgoing

    total_in_arcs = sum(len(v) for v in source_arcs_ingoing.values())
    print(total_in_arcs, "ingoing to source nodes in total")

    total_out_arcs = sum(len(v) for v in source_arcs_outgoing.values())
    print(total_out_arcs, "outgoing from source nodes in total")

    return source_arcs_ingoing, source_arcs_outgoing

### Exit arcs

In [13]:
def build_exit_arcs(V_exit_nodeIDs, A):
    """
    Build dictionaries of outgoing and ingoing arcs for each exit node.
    """
    exit_arcs_ingoing = {}
    exit_arcs_outgoing = {}

    for exitnode in V_exit_nodeIDs:
        ingoing = set()
        outgoing = set()
        for arc in A:
            u, v = split_edge_to_nodes(arc)  # unpack arc from string
            if u == exitnode:
                outgoing.add(arc)
            if v == exitnode:
                ingoing.add(arc)
        # assign sets to dictionaries
        exit_arcs_ingoing[exitnode] = ingoing
        exit_arcs_outgoing[exitnode] = outgoing

    total_in_arcs = sum(len(v) for v in exit_arcs_ingoing.values())
    total_out_arcs = sum(len(v) for v in exit_arcs_outgoing.values())
    return exit_arcs_ingoing, total_in_arcs, exit_arcs_outgoing, total_out_arcs

### Transit arcs

In [14]:
def build_transitnode_arcs(V_transit, A):
    """
    Build ingoing/outgoing arc dictionaries for transit nodes,
    where arcs are strings like 'node1node245'.
    
    Parameters:
        V_transit (iterable): Transit node IDs, e.g., ['node5', 'node8', ...]
        A (iterable): Arcs as strings, e.g., 'node1node245'
    
    Returns:
        tuple: (ingoing_per_transitnode, outgoing_per_transitnode, total_ingoing, total_outgoing)
    """
    ingoing_per_transitnode = {}
    outgoing_per_transitnode = {}

    for tnode in V_transit:
        ingoing = {arc for arc in A if split_edge_to_nodes(arc)[1] == tnode}
        outgoing = {arc for arc in A if split_edge_to_nodes(arc)[0] == tnode}
        ingoing_per_transitnode[tnode] = ingoing
        outgoing_per_transitnode[tnode] = outgoing

    total_ingoing = sum(len(v) for v in ingoing_per_transitnode.values())
    total_outgoing = sum(len(v) for v in outgoing_per_transitnode.values())

    return ingoing_per_transitnode, outgoing_per_transitnode, total_ingoing, total_outgoing

## Load parameters and actual values

### 💰 Load Costs

In [15]:
def load_and_process_edge_attributes(csv_path, reverse_mapping):
    """
    Load edge attribute CSV and create EdgeID using node reverse mapping.

    Parameters:
        csv_path (str): Path to the CSV file containing edge attributes.
        reverse_mapping (dict): Mapping from (x, y) coordinates to node IDs.

    Returns:
        pd.DataFrame: Edge attribute DataFrame with added 'node1_ID', 'node2_ID', and 'EdgeID' columns.
    """
    # Load CSV
    edge_attr_df = pd.read_csv(csv_path)

    # Map Node1 and Node2 coordinates to node IDs
    edge_attr_df['node1_ID'] = edge_attr_df['Source(x,y)'].map(reverse_mapping)
    edge_attr_df['node2_ID'] = edge_attr_df['Target(x,y)'].map(reverse_mapping)

    # Ensure both are strings and create EdgeID
    edge_attr_df['EdgeID'] = edge_attr_df['node1_ID'].astype(str) + edge_attr_df['node2_ID'].astype(str)

    return edge_attr_df


### Set costs parameters

In [16]:
def build_cost_dicts(E, W, edge_attr_df):
    """
    Build cost dictionaries for construction, maintenance, and upgrade.

    Parameters:
        E (iterable): Collection of edge IDs.
        W (list/tuple): List of weight categories (e.g., [5, 10]).
        edge_attr_df (pd.DataFrame): DataFrame with columns:
            'EdgeID', 'Build5m', 'Build10m', 'Maintain5m', 'Maintain10m', 'Upgrade'.

    Returns:
        tuple:
            CostC (dict): {(edge, weight): construction cost}.
            CostM (dict): {(edge, weight): maintenance cost}.
            CostU (dict): {edge: upgrade cost}.
    """
    # Initialize dictionaries
    CostC = {(e, w): None for e in E for w in W}
    CostM = {(e, w): None for e in E for w in W}
    CostU = {e: None for e in E}

    # prepare the edge attributed df
    edge_attr_df['node1_ID'] = edge_attr_df['Source(x,y)'].map(reverse_mapping)
    edge_attr_df['node2_ID'] = edge_attr_df['Target(x,y)'].map(reverse_mapping)
    edge_attr_df['EdgeID'] = edge_attr_df['node1_ID'].astype(str) + edge_attr_df['node2_ID'].astype(str)

    # Assign actual cost values from DataFrame
    for _, row in edge_attr_df.iterrows():
        e = row['EdgeID']

        CostC[(e, W[0])] = round(row['Build5m'], 2)
        CostC[(e, W[1])] = round(row['Build10m'], 2)

        CostM[(e, W[0])] = round(row['Maintain5m'], 2)
        CostM[(e, W[1])] = round(row['Maintain10m'], 2)

        CostU[e] = round(row['Upgrade'], 2)

    return CostC, CostM, CostU

### Load Access Needs

In [17]:
def load_access_needs(csv_path, filter_stands=None, split_columns=True, n_split=5):
    """
    Load a stands access needs CSV, set 'ID_UG' as the index, optionally filter by a set of stand IDs,
    modify column names to integers, and optionally split into two parts.

    Parameters:
        csv_path (str): Path to the CSV file.
        filter_stands (set, optional): Set of stand IDs (ID_UG) to keep. Defaults to None.
        split_columns (bool, optional): Whether to split the dataframe into two parts. Defaults to True.
        n_split (int, optional): Number of columns for the split. Defaults to 5.

    Returns:
        tuple: (df1, df2) if split_columns is True, else df
            - df1: First n_split columns (original)
            - df2: Last n_split columns (wide/extended)
    """
    # Load CSV and set index
    df = pd.read_csv(csv_path)
    df.set_index('ID_UG', inplace=True)

    # Filter for given stand IDs
    if filter_stands is not None:
        df = df[df.index.isin(filter_stands)]

    # Modify column names to integers based on suffix after '_'
    df.columns = df.columns.str.split('_').str[-1].astype(int)

    # Optionally split into two parts
    if split_columns:
        df1 = df.iloc[:, :n_split]  # First n_split columns
        df2 = df.iloc[:, -n_split:]  # Last n_split columns
        return df1, df2

    return df

### Set needroad parameters, for all stands ID in S
$$needroad_{s,t}^\text{timber}$$

$$needroad_{s,t}^\text{wide}$$

In [18]:
def set_needroad_parameters(S, T, accessneeds_df_dict):
    """
    Create a dictionary of need road parameters for different access types.

    Parameters:
        S (iterable): Source nodes.
        T (iterable): Target nodes.
        accessneeds_df_dict (dict): Mapping {access_type: DataFrame} with needs from sources to targets.

    Returns:
        dict: Dictionary with keys (access_type, s, t) and corresponding need values.
    """
    needroad = {}
    for access_type, df in accessneeds_df_dict.items():
        for s in S:
            for t in T:
                needroad[(access_type, s, t)] = df.loc[s, t] if s in df.index else 0
    return needroad

### Calculate maxflow parameters
$$maxflow_t^\text{timber} = \sum_{s\in S}needroad_{s,t}^\text{timber}$$

In [19]:
def set_maxflow_params(S, T, needroad_w_s_t, access_types):
    """
    Create maxflow dictionaries for given access types.

    Parameters:
        S (iterable): Source nodes.
        T (iterable): Target nodes.
        needroad_w_s_t (dict): Dictionary {(access_type, s, t): need}.
        access_types (list): List of access types to compute ('timber', 'wide', etc.).

    Returns:
        dict: Dictionary of maxflow dicts for each access type, e.g.,
              {'timber': maxflow_timber_dict, 'wide': maxflow_wide_dict}
    """
    maxflow_dicts = {}
    
    for w in access_types:
        maxflow = {
            (w, t): sum(needroad_w_s_t[(w, s, t)] for s in S for t in T)
            for t in T
        }
        maxflow_dicts[w] = maxflow
    
    return maxflow_dicts

$$maxflow_t^\text{wide} = \sum_{s\in S}needroad_{s,t}^\text{wide}$$

## Define binary decision variables

### define binary variables for construction maintenance upgrade

In [20]:
def create_binary_decision_variables(E, W, T):
    """
    Create dictionaries of binary decision variable names for construction, maintenance, and upgrade.

    Parameters:
        E (iterable): Edge IDs.
        W (iterable): Weight categories (e.g., [5, 10]).
        T (iterable): Time steps (integers).

    Returns:
        tuple:
            C (dict): {(edge, weight, t): variable_name} for construction.
            M (dict): {(edge, weight, t): variable_name} for maintenance.
            U (dict): {(edge, t): variable_name} for upgrade.
    """
    C = {(e, w, t): f"C_{w}_{e}_t{t}" for e in E for w in W for t in (0,) + tuple(T)}
    M = {(e, w, t): f"M_{w}_{e}_t{t}" for e in E for w in W for t in (0,) + tuple(T)}
    U = {(e, t): f"U_{e}_t{t}" for e in E for t in (0,) + tuple(T)}

    print("Decision variables for C, M, U created.")
    return C, M, U


### verify the created decision variables

## Define integer variables for flow

In [21]:
def create_integer_flow_variables(A, W, T):
    """
    Create dictionary of integer flow variable names.

    Parameters:
        A (iterable): List of arcs.
        W (iterable): Weight categories.
        T (iterable): Time steps.

    Returns:
        dict: {(arc, weight, t): variable_name}
    """
    flow = {(a, w, t): f"flow_{w}_{a}_t{t}" for a in A for w in W for t in T}
    return flow

## Build model

### Add binary decision variables

In [22]:
def add_binary_variables_to_model(model, C, M, U, CostC, CostM, CostU):
    """
    Add binary decision variables (C, M, U) to a CPLEX model and prepare objective terms and coefficients.

    Parameters:
        model: CPLEX model object.
        C, M, U (dict): Binary variable dictionaries {(edge, weight, t) or (edge, t): name}.
        CostC, CostM, CostU (dict): Cost dictionaries corresponding to the variables.

    Returns:
        tuple: (objective_terms, objective_coeffs)
            - objective_terms: list of variable names
            - objective_coeffs: list of corresponding cost coefficients
    """
    objective_terms = []
    objective_coeffs = []

    # Add C_vars
    for (e, w, t), name in C.items():
        lb, ub = (0, 0) if t == 0 else (0, 1)
        model.variables.add(names=[name], lb=[lb], ub=[ub], types=["B"])
        objective_terms.append(name)
        objective_coeffs.append(CostC[(e, w)])

    # Add M_vars
    for (e, w, t), name in M.items():
        lb, ub = (0, 0) if t == 0 else (0, 1)
        model.variables.add(names=[name], lb=[lb], ub=[ub], types=["B"])
        objective_terms.append(name)
        objective_coeffs.append(CostM[(e, w)])

    # Add U_vars
    for (e, t), name in U.items():
        lb, ub = (0, 0) if t == 0 else (0, 1)
        model.variables.add(names=[name], lb=[lb], ub=[ub], types=["B"])
        objective_terms.append(name)
        objective_coeffs.append(CostU[e])
    
    print("Decision variables for C, M, U added to model.")

    return objective_terms, objective_coeffs

#### verify & store
should be len(E) x (len (T)+1) x 5
because 5 different actions

In [23]:
def verify_binary_variables(model, E, T, objective_coeffs, n_actions=5):
    """
    Verify that the number of action variables and objective coefficients is correct.

    Parameters:
        model: CPLEX model object.
        E (iterable): List of edges.
        T (iterable): List of time steps.
        objective_coeffs (list): List of objective function coefficients.
        n_actions (int): Number of different actions per edge-time (default 5).

    Prints:
        Verification messages for variable count and objective coefficients.
    """
    n_action_variables = len(model.variables.get_names())
    expected_vars = len(E) * (len(T)+1) * n_actions

    print(n_action_variables, "action variables")
    print(f"{n_actions}*len(E)*(len(T)+1) =", expected_vars)
    print(len(objective_coeffs), "objective coefficients")

    if n_action_variables == expected_vars:
        print("✅ Verification passed: variable count match expected")
    else:
        print("❌ Verification failed: variable count does not match expected")
    
    if len(objective_coeffs) == expected_vars:
        print("✅ Verification passed: coefficient count match expected")
    else:
        print("❌ Verification failed: coefficient count does not match expected")

    return n_action_variables

def save_binary_variables_info(model, CostC, CostM, CostU, model_dir):
    """
    Save information about binary decision variables to a text file.

    Parameters:
        model: CPLEX model object.
        CostC, CostM, CostU (dict): Cost dictionaries for construction, maintenance, and upgrade.
        model_dir (str): Directory where the file will be saved.
    """
    os.makedirs(model_dir, exist_ok=True)
    file_path = os.path.join(model_dir, "binary_decision_variables.txt")

    with open(file_path, "w") as f:
        var_names = model.variables.get_names()
        for name in var_names:
            lb = model.variables.get_lower_bounds([name])[0]
            ub = model.variables.get_upper_bounds([name])[0]
            var_type = model.variables.get_types([name])[0]

            # Determine edge/arc and weight
            e_match = re.search(r"node[\w]+(?=_t\d+)", name)
            e = e_match.group(0) if e_match else "N/A"

            if name[0] != "U":
                w_match = re.search(r"[CMU]_(\w+)(?=_node)", name)
                w = w_match.group(1) if w_match else "N/A"
                if name[0] == "C":
                    cost_coeff = CostC.get((e, w), "N/A")
                else:
                    cost_coeff = CostM.get((e, w), "N/A")
            else:
                cost_coeff = CostU.get(e, "N/A")

            f.write(f"{name}: lb={lb}, ub={ub}, type={var_type}, cost={cost_coeff}\n")

    print(f"Binary variable info saved to {file_path}")

In [24]:
def create_objective_dataframe(objective_terms, objective_coeffs):
    """
    Create a pandas DataFrame with variable names and corresponding objective costs.

    Parameters:
        objective_terms (list): List of variable names.
        objective_coeffs (list): Corresponding cost coefficients.

    Returns:
        pd.DataFrame: DataFrame with columns ['variable name', 'Cost'].
    """
    obj_df = pd.DataFrame({
        'variable name': objective_terms,
        'Cost': objective_coeffs
    })
    return obj_df


### Objective Function

In [25]:
def set_objective(model, objective_terms, objective_coeffs):

    # Assign linear objective
    model.objective.set_linear(list(zip(objective_terms, objective_coeffs)))

    model.objective.set_sense(model.objective.sense.minimize) 
    # Confirm action
    print(f"Linear objective function assigned and set to minimize.")

#### store objective function

In [26]:
def save_objective_function_to_file(model, file_path):
    """
    Save the linear objective function of a CPLEX model to a text file.

    Parameters:
        model: CPLEX model object.
        file_path (str): Full path to the text file where the objective will be saved.
    """
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    objective_expr = model.objective.get_linear()
    
    with open(file_path, "w") as f:
        for i, coeff in enumerate(objective_expr):
            var_name = model.variables.get_names(i)
            f.write(f"{coeff} * {var_name}\n")
    
    print(f"Objective function saved to {file_path}")

### Add flow variables

In [27]:
def add_flow_variables(model, flow, lb=0, ub=9):
    """
    Add integer flow variables to a CPLEX model.

    Parameters:
        model: CPLEX model object.
        flow (dict): Dictionary of flow variables {(arc, weight, t): name}.
        lb (int, optional): Lower bound for variables. Default is 0.
        ub (int, optional): Upper bound for variables. Default is 9.
    """
    for (e, w, t), name in flow.items():
        model.variables.add(names=[name], lb=[lb], ub=[ub], types=["I"])
    
    print(f"{len(flow)} flow variables added to the model.")


##### verify adding of flow variables
should be 2 x len(E) x len(T) variables

In [28]:
def verify_flow_variables(model, n_action_variables, A, W, T):
    """
    Verify the number of flow variables in a CPLEX model.

    Parameters:
        model: CPLEX model object.
        n_action_variables (int): Number of previously added action variables.
        A (iterable): List of arcs.
        W (iterable): List of weight categories.
        T (iterable): List of time steps.
    """
    n_variables = len(model.variables.get_names())
    n_flow_variables = n_variables - n_action_variables
    expected_flow_variables = len(A) * len(T) * len(W)

    print(n_variables, "total variables")
    print(n_flow_variables, "flow variables")
    print(f"Expected flow variables (len(A)*len(T)*len(W)) = {expected_flow_variables}")

    if n_flow_variables == expected_flow_variables:
        print("✅ Flow variable count matches expected")
    else:
        print("❌ Flow variable count does NOT match expected")

    return n_flow_variables

In [ ]:
def save_flow_variables_info(model, n_flow_variables, save_dir):
    """
    Save information about flow variables to a text file.

    Parameters:
        model: CPLEX model object.
        n_flow_variables (int): Number of flow variables to consider (from the end of variable list).
        save_dir (str): Full path to the output text file.
    """
    os.makedirs(save_dir, exist_ok=True)
    file_path = os.path.join(save_dir, "integer_flow_variables.txt")

    var_names = model.variables.get_names()[-n_flow_variables:]

    with open(file_path, "w") as f:
        for name in var_names:
            lb = model.variables.get_lower_bounds([name])[0]
            ub = model.variables.get_upper_bounds([name])[0]
            var_type = model.variables.get_types([name])[0]
            f.write(f"{name}: lb={lb}, ub={ub}, type={var_type}\n")

    print(f"Flow variable info saved to {file_path}")


In [30]:
def verify_transitnode_arcs(flow, ingoing_per_transitnode, outgoing_per_transitnode, verbose=True):
    """
    Verifies that all arcs referenced in ingoing/outgoing transit nodes exist in the flow variables.

    Parameters
    ----------
    flow : dict
        Dictionary of flow variables indexed by (arc, road_type, period).
    ingoing_per_transitnode : dict
        Dictionary mapping transit nodes to lists of ingoing arcs.
    outgoing_per_transitnode : dict
        Dictionary mapping transit nodes to lists of outgoing arcs.
    verbose : bool, default True
        Whether to print detailed results.

    Returns
    -------
    dict
        Dictionary containing sets of missing arcs: {'missing_in': set, 'missing_out': set}
    """
    # Set of all arcs in the flow variables
    A_for_flow = {a for (a, _, _) in flow.keys()}

    # Find missing arcs
    missing_in_arcs = {
        a for arcs in ingoing_per_transitnode.values() 
        for a in arcs 
        if a not in A_for_flow
    }
    missing_out_arcs = {
        a for arcs in outgoing_per_transitnode.values() 
        for a in arcs 
        if a not in A_for_flow
    }

    if verbose:
        print("\n=== Transit Node Arc Verification ===")
        print(f"Missing arcs in ingoing per transit node: {len(missing_in_arcs)}")
        if missing_in_arcs:
            print(sorted(missing_in_arcs))
        print(f"Missing arcs in outgoing per transit node: {len(missing_out_arcs)}")
        if missing_out_arcs:
            print(sorted(missing_out_arcs))

        if not missing_in_arcs and not missing_out_arcs:
            print("✅ All transit node arcs are properly represented in the flow variables.")
        else:
            print("⚠️ Some arcs are missing. See above.")

    return {"missing_in": missing_in_arcs, "missing_out": missing_out_arcs}


## Add Constraints

### Flow (wide) enforces road existence (wide)

$$\forall t \in T \quad \forall e\in E: \qquad
flow_{(u,v),t}^{\text{wide}} + flow_{(v,u),t}^{\text{wide}}  \leq maxflow_t^{\text{wide}} \big(C _{e,t}^{\text{wide}} + M_{e,t}^{\text{wide}} +  U_{e,t}\big)$$
$$\Leftrightarrow$$
$$flow_{(u,v),t}^{\text{wide}} + flow_{(v,u),t}^{\text{wide}} - maxflow_t^{\text{wide}} \big(C _{e,t}^{\text{wide}} + M_{e,t}^{\text{wide}} +  U_{e,t}\big)  \leq 0$$
$$\Leftrightarrow$$
$$
flow_{(u,v),t}^{\text{wide}} + flow_{(v,u),t}^{\text{wide}} - maxflow_t^{\text{wide}}C _{e,t}^{\text{wide}} - maxflow_t^{\text{wide}}M_{e,t}^{\text{wide}} - maxflow_t^{\text{wide}}U_{e,t} \leq 0\\
$$

In [31]:
def add_wide_flow_constraints(model, flow, C, M, U, maxflow_w_t, fw_to_bw_map, T, A_fw, road_type='wide', verification_log=None):
    """
    Adds constraints to enforce that flow only occurs on wide roads and optionally logs verification data.

    Prints a single checkmark when all constraints are added.
    """
    if verification_log is None:
        verification_log = []

    for t in T:
        for a_fw in A_fw:
            a_bw = fw_to_bw_map[a_fw]
            e = a_fw
            try:
                expr_vars = [flow[(a_fw, road_type, t)],
                             flow[(a_bw, road_type, t)],
                             C[(e, road_type, t)],
                             M[(e, road_type, t)],
                             U[(e, t)]]
                
                coeff = int(maxflow_w_t[(road_type, t)])
                expr_coeffs = [1, 1, -coeff, -coeff, -coeff]
                
                constraint_name = f"Constraint_flow_enforcing_road_{road_type}_{e}_{t}"
                
                # Add constraint
                model.linear_constraints.add(
                    lin_expr=[[expr_vars, expr_coeffs]],
                    senses=["L"],
                    rhs=[0],
                    names=[constraint_name]
                )
                
                # Store verification data
                verification_log.append({
                    "constraint_name": constraint_name,
                    "variables": expr_vars,
                    "coefficients": expr_coeffs
                })
                
            except KeyError as ke:
                verification_log.append({
                    "constraint_name": f"Missing_{road_type}_{e}_{t}",
                    "error": str(ke)
                })
                print("problem")

    # Single confirmation after all constraints are added
    print("Done.")
    return verification_log


### [helper function] verify and store constraints

In [32]:
def verify_and_store_constraints(model, verification_data, group_name, expected_count, output_file):
    """
    Verify constraints for a specific group, and store them in a file.

    Parameters:
        model: CPLEX model object.
        verification_data (list of dict): Log of constraints, each with "constraint_name".
        group_name (str): Identifier substring for the constraint group.
        expected_count (int): Expected number of constraints in this group.
        output_file (str): Path to store constraints belonging to this group.

    Returns:
        bool: True if the check passed, False otherwise.
    """
    # Count total constraints in the model
    total_constraints = model.linear_constraints.get_num()

    # Filter constraints of this group
    group_constraints = [
        v for v in verification_data
        if group_name in v.get("constraint_name", "")
    ]
    n_constraints_group = len(group_constraints)

    # --- Report ---
    print(f"\n=== Constraint Verification: {group_name} ===")
    print(f"Total constraints in model: {total_constraints}")
    print(f"Constraints added in this group: {n_constraints_group}")
    print(f"Expected number of constraints for this group: {expected_count}")

    if n_constraints_group == expected_count:
        print(f"✅ Check passed: {n_constraints_group} constraints added as expected.")
        status = True
    else:
        print(f"❌ Check failed: {n_constraints_group} constraints added, expected {expected_count}.")
        status = False

    # --- Store constraints in file ---
    with open(output_file, "w") as f:
        f.write(f"Constraint group: {group_name}\n")
        f.write(f"Total in this group: {n_constraints_group}\n")
        f.write(f"Expected: {expected_count}\n\n")
        f.write("Constraints:\n")
        for c in group_constraints:
            f.write(str(c) + "\n")

    print(f"Constraints written to: {output_file}")

should be 5 x len(E) constraints

### Flow (timber) enforcing road existence

$$
flow_{(u,v),t}^{\text{timber}} + flow_{(v,u),t}^{\text{timber}}\leq maxflow_{t}^{\text{timber}} \big(C_{e,t}^{\text{timber}}+ M_{e,t}^{\text{timber}} + C _{e,t}^{\text{wide}} + M_{e,t}^{\text{wide}} +  U_{e,t}\big)$$
$$
\Leftrightarrow$$
$$flow_{(u,v),t}^{\text{timber}} + flow_{(v,u),t}^{\text{timber}}- maxflow_{t}^{\text{timber}} \big(C_{e,t}^{\text{timber}}+ M_{e,t}^{\text{timber}} + C _{e,t}^{\text{wide}} + M_{e,t}^{\text{wide}} +  U_{e,t}\big) \leq 0$$
$$\Leftrightarrow$$
$$flow_{(u,v),t}^{\text{timber}} + flow_{(v,u),t}^{\text{timber}}- maxflow_{t}^{\text{timber}} C_{e,t}^{\text{timber}}-maxflow_{t}^{\text{timber}}  M_{e,t}^{\text{timber}} -maxflow_{t}^{\text{timber}}  C _{e,t}^{\text{wide}} -maxflow_{t}^{\text{timber}}  M_{e,t}^{\text{wide}} -maxflow_{t}^{\text{timber}}  U_{e,t} \leq 0$$

In [33]:
def add_timber_flow_constraints(model, flow, C, M, U, maxflow_w_t, fw_to_bw_map, T, E, road_type='timber', verification_log=None):
    """
    Adds constraints to enforce that flow only occurs on timber roads (or at least timber roads).

    Stores verification data instead of printing each constraint individually.
    Prints a single confirmation after all constraints are added.
    """
    if verification_log is None:
        verification_log = []

    for t in T:
        for a_fw in E:
            a_bw = fw_to_bw_map[a_fw]
            e = a_fw

            # Validate keys exist (fail fast if missing)
            assert (a_fw, 'timber', t) in flow, f"Missing flow var: {(a_fw, 'timber', t)}"
            assert (a_bw, 'timber', t) in flow, f"Missing flow var: {(a_bw, 'timber', t)}"
            assert (e, 'timber', t) in C, f"Missing C var: {(e, 'timber', t)}"
            assert (e, 'timber', t) in M, f"Missing M var: {(e, 'timber', t)}"
            assert (e, 'wide', t) in C, f"Missing C var: {(e, 'wide', t)}"
            assert (e, 'wide', t) in M, f"Missing M var: {(e, 'wide', t)}"
            assert (e, t) in U, f"Missing U var: {(e, t)}"
            assert ('timber', t) in maxflow_w_t, f"Missing maxflow param: {('timber', t)}"

            # Define linear expression variables
            expr_vars = [
                flow[(a_fw, 'timber', t)],
                flow[(a_bw, 'timber', t)],
                C[(e, 'timber', t)],
                M[(e, 'timber', t)],
                C[(e, 'wide', t)],
                M[(e, 'wide', t)],
                U[(e, t)]
            ]

            coeff = int(maxflow_w_t[('timber', t)])
            expr_coeffs = [1, 1, -coeff, -coeff, -coeff, -coeff, -coeff]

            constraint_name = f"Constraint_timberflow_enforcing_road_{e}_{t}"

            # Add the constraint
            model.linear_constraints.add(
                lin_expr=[[expr_vars, expr_coeffs]],
                senses=["L"],
                rhs=[0],
                names=[constraint_name]
            )

            # Store verification data
            verification_log.append({
                "constraint_name": constraint_name,
                "variables": expr_vars,
                "coefficients": expr_coeffs
            })

    # Single confirmation
    print("Done.")
    return verification_log

should be 5 x len(E) constraints

### Flow conservation

$$\forall t \in T \quad \forall v \in V_{\text{transit}}\quad \forall w \in W$$
$
\begin{align}
       \sum_{u \in N(v)} flow_{(u,v),t}^{w} &= \sum_{u \in N(v)} flow_{(v,u),t}^{w}\\
       &\Leftrightarrow
       \\
       \sum_{u \in N(v)} flow_{(u,v),t}^{w} - \sum_{u \in N(v)} flow_{(v,u),t}^{w}&=0
\end{align}
$

In [34]:
def add_flow_conservation_constraints(model, flow, ingoing_per_transitnode, outgoing_per_transitnode, W, T, verification_log=None):
    """
    Adds flow conservation constraints for all transit nodes, road types, and periods.

    Parameters
    ----------
    model : CPLEX model
        Optimization model to which constraints are added.
    flow : dict
        Dictionary of flow variables indexed by (arc, road_type, period).
    ingoing_per_transitnode : dict
        Mapping transit nodes to lists of ingoing arcs.
    outgoing_per_transitnode : dict
        Mapping transit nodes to lists of outgoing arcs.
    W : iterable
        Road types.
    T : iterable
        Periods.
    verification_log : list, optional
        List to store verification data (variables, coefficients, constraint names).

    Returns
    -------
    list
        Updated verification_log with new entries.
    """
    if verification_log is None:
        verification_log = []

    for w in W:
        for t in T:
            for tnode, in_arcs in ingoing_per_transitnode.items():
                out_arcs = outgoing_per_transitnode.get(tnode, [])

                # Validate that all variables exist
                for a_in in in_arcs:
                    assert (a_in, w, t) in flow, f"Missing flow variable: {(a_in, w, t)}"
                for a_out in out_arcs:
                    assert (a_out, w, t) in flow, f"Missing flow variable: {(a_out, w, t)}"

                # Linear expression
                expr_vars = [flow[(a_in, w, t)] for a_in in in_arcs] + [flow[(a_out, w, t)] for a_out in out_arcs]
                expr_coeffs = [1] * len(in_arcs) + [-1] * len(out_arcs)

                constraint_name = f"Constraint_flow_conservation_{w}_{t}_{tnode}"

                # Add constraint
                model.linear_constraints.add(
                    lin_expr=[[expr_vars, expr_coeffs]],
                    senses=["E"],  # equality
                    rhs=[0],
                    names=[constraint_name]
                )

                # Store verification data
                verification_log.append({
                    "constraint_name": constraint_name,
                    "variables": expr_vars,
                    "coefficients": expr_coeffs
                })

    print(f"Done.")
    return verification_log


should be 2 x len(V_transit) x 5

### Outflow if and only if stand needs road

$\begin{align}
\forall t \in T\quad\forall s\in V_{source}\quad\forall w\in W:\quad\\
   \sum_{v\in B(s)}flow_{(s,v),t}^{w} &= need\_road_{s,t}^{w}
\end{align}
$

In [35]:
def add_outflow_needroad_constraints(model, flow, needroad_w_s_t, source_arcs_outgoing, W, T, verification_log=None):
    """
    Adds constraints to enforce that outflow occurs only from stands that need a road.

    Parameters
    ----------
    model : CPLEX model
        Optimization model to which constraints are added.
    flow : dict
        Dictionary of flow variables indexed by (arc, road_type, period).
    needroad_w_s_t : dict
        Dictionary of expected outflow from each stand: keyed by (road_type, stand, period).
    source_arcs_outgoing : dict
        Mapping from stand s to list of outgoing arcs.
    W : iterable
        Road types.
    T : iterable
        Periods.
    verification_log : list, optional
        List to store verification data (variables, coefficients, RHS, constraint names).

    Returns
    -------
    list
        Updated verification_log with new entries.
    """
    if verification_log is None:
        verification_log = []

    for w in W:
        for t in T:
            for s, out_arcs in source_arcs_outgoing.items():
                # Validate keys exist
                for e in out_arcs:
                    assert (e, w, t) in flow, f"Missing flow variable: {(e, w, t)}"
                assert (w, s, t) in needroad_w_s_t, f"Missing needroad parameter: {(w, s, t)}"

                # Define linear expression
                expr_vars = [flow[(e, w, t)] for e in out_arcs]
                expr_coeffs = [1] * len(out_arcs)
                rhs = int(needroad_w_s_t[(w, s, t)])

                constraint_name = f"Constraint_outflow_iff_needroad_{w}_{s}_t{t}"

                # Add constraint
                model.linear_constraints.add(
                    lin_expr=[[expr_vars, expr_coeffs]],
                    senses=["E"],  # equality
                    rhs=[rhs],
                    names=[constraint_name]
                )

                # Store verification data
                verification_log.append({
                    "constraint_name": constraint_name,
                    "variables": expr_vars,
                    "coefficients": expr_coeffs,
                    "rhs": rhs
                })

    # Single confirmation
    print(f"Done.")
    return verification_log

should be len(V_source) x 5 actions x 2

### No Inflow to source nodes

$\begin{align}
\forall t \in T\quad\forall s\in V_{source}:\quad
    \sum_{u\in B(s)} \Big(flow_{(u,s),t}^{\text{wide}} + flow_{(u,s),t}^{\text{timber}}\Big) = 0
\end{align}
$

In [36]:
def add_no_inflow_source_constraints(model, flow, source_arcs_ingoing, T, verification_log=None):
    """
    Adds constraints to prevent inflow to source nodes.

    Parameters
    ----------
    model : CPLEX model
        Optimization model to which constraints are added.
    flow : dict
        Dictionary of flow variables indexed by (arc, road_type, period).
    source_arcs_ingoing : dict
        Mapping from source node s to list of ingoing arcs.
    T : iterable
        Periods.
    verification_log : list, optional
        List to store verification data (variables, coefficients, constraint names).

    Returns
    -------
    list
        Updated verification_log with new entries.
    """
    if verification_log is None:
        verification_log = []

    for t in T:
        for s, in_arcs in source_arcs_ingoing.items():
            # Validate keys exist
            for a in in_arcs:
                assert (a, 'timber', t) in flow, f"Missing flow variable: {(a, 'timber', t)}"
                assert (a, 'wide', t) in flow, f"Missing flow variable: {(a, 'wide', t)}"

            # Linear expression: sum of all inflows (timber + wide)
            expr_vars = [flow[(a, 'timber', t)] for a in in_arcs] + [flow[(a, 'wide', t)] for a in in_arcs]
            expr_coeffs = [1] * len(in_arcs) + [1] * len(in_arcs)

            constraint_name = f"Constraint_no_inflow_source_{s}_{t}"

            # Add constraint
            model.linear_constraints.add(
                lin_expr=[[expr_vars, expr_coeffs]],
                senses=["E"],  # equality
                rhs=[0],
                names=[constraint_name]
            )

            # Store verification data
            verification_log.append({
                "constraint_name": constraint_name,
                "variables": expr_vars,
                "coefficients": expr_coeffs
            })

    # Single confirmation
    print(f"Done.")
    return verification_log

should be len(V_source)*5

### Maintenance & Upgrade Constraints for timber roads

$$M_{e,t}^{\text{timber}} + U_{e,t} \leq C_{e,t-1}^{\text{timber}} + M_{e,t-1}^{\text{timber}}$$
$$\Leftrightarrow$$
$$M_{e,t}^{\text{timber}} + U_{e,t} - C_{e,t-1}^{\text{timber}} - M_{e,t-1}^{\text{timber}} \leq 0$$


In [37]:
def add_maintain_upgrade_constraints(model, C, M, U, E, T, road_type='timber', verification_log=None):
    """
    Adds constraints to ensure Maintenance or Upgrade on timber roads respects previous construction/maintenance.

    Parameters
    ----------
    model : CPLEX model
        Optimization model to which constraints are added.
    C : dict
        Construction variables indexed by (edge, road_type, period).
    M : dict
        Maintenance variables indexed by (edge, road_type, period).
    U : dict
        Upgrade variables indexed by (edge, period).
    E : iterable
        List of edges.
    T : iterable
        List of periods.
    road_type : str, default 'timber'
        Type of road for the constraint.
    verification_log : list, optional
        List to store verification data (variables, coefficients, constraint names).

    Returns
    -------
    list
        Updated verification_log with new entries.
    """
    if verification_log is None:
        verification_log = []

    for t in T:
        for e in E:
            # Skip t=0 if you don’t want negative indices
            if t == 0:
                continue

            # Validate keys exist
            assert (e, road_type, t) in M, f"Missing M variable: {(e, road_type, t)}"
            assert (e, t) in U, f"Missing U variable: {(e, t)}"
            assert (e, road_type, t-1) in C, f"Missing C variable: {(e, road_type, t-1)}"
            assert (e, road_type, t-1) in M, f"Missing M variable: {(e, road_type, t-1)}"

            # Linear expression
            expr_vars = [
                M[(e, road_type, t)],
                U[(e, t)],
                C[(e, road_type, t-1)],
                M[(e, road_type, t-1)]
            ]
            expr_coeffs = [1, 1, -1, -1]

            constraint_name = f"Constraint_maintain_upgrade_{road_type}_{e}_{t}"

            # Add constraint
            model.linear_constraints.add(
                lin_expr=[[expr_vars, expr_coeffs]],
                senses=["L"],  # <=
                rhs=[0],
                names=[constraint_name]
            )

            # Store verification data
            verification_log.append({
                "constraint_name": constraint_name,
                "variables": expr_vars,
                "coefficients": expr_coeffs
            })

    # Single confirmation
    print(f"Done for '{road_type}'.")
    return verification_log


should be 5x len(E)

### Maintenance/ Upgrade constraints for wide roads

$\begin{align}
M_{e,t}^{wide} &\leq C_{e,t-1}^{wide} + M_{e,t-1}^{wide} + U_{e,t-1}\\
\\
&\Leftrightarrow\\\
\\
M_{e,t}^{wide} - C_{e,t-1}^{wide} - M_{e,t-1}^{wide} - U_{e,t-1} &\leq 0
\end{align}
$

In [38]:
def add_maintain_upgrade_wide_constraints(model, C, M, U, E, T, road_type='wide', verification_log=None):
    """
    Adds constraints to ensure Maintenance or Upgrade on wide roads respects previous construction/maintenance.

    Parameters
    ----------
    model : CPLEX model
        Optimization model to which constraints are added.
    C : dict
        Construction variables indexed by (edge, road_type, period).
    M : dict
        Maintenance variables indexed by (edge, road_type, period).
    U : dict
        Upgrade variables indexed by (edge, period).
    E : iterable
        List of edges.
    T : iterable
        List of periods.
    road_type : str, default 'wide'
        Type of road for the constraint.
    verification_log : list, optional
        List to store verification data (variables, coefficients, constraint names).

    Returns
    -------
    list
        Updated verification_log with new entries.
    """
    if verification_log is None:
        verification_log = []

    for t in T:
        for e in E:
            # Skip t=0 if needed
            if t == 0:
                continue

            # Validate all required keys exist
            assert (e, road_type, t) in M, f"Missing M variable: {(e, road_type, t)}"
            assert (e, road_type, t-1) in C, f"Missing C variable: {(e, road_type, t-1)}"
            assert (e, road_type, t-1) in M, f"Missing M variable: {(e, road_type, t-1)}"
            assert (e, t-1) in U, f"Missing U variable: {(e, t-1)}"

            # Linear expression
            expr_vars = [
                M[(e, road_type, t)],
                C[(e, road_type, t-1)],
                M[(e, road_type, t-1)],
                U[(e, t-1)]
            ]
            expr_coeffs = [1, -1, -1, -1]

            constraint_name = f"Constraint_maintain_upgrade_{road_type}_{e}_{t}"

            # Add constraint
            model.linear_constraints.add(
                lin_expr=[[expr_vars, expr_coeffs]],
                senses=["L"],  # <=
                rhs=[0],
                names=[constraint_name]
            )

            # Store verification data
            verification_log.append({
                "constraint_name": constraint_name,
                "variables": expr_vars,
                "coefficients": expr_coeffs
            })

    print(f"Done for wide.")
    return verification_log


### Inspect the result of adding constraints

In [39]:
def inspect_constraints(model, output_file=None, return_df=False):
    """
    Inspect all linear constraints in a CPLEX model and optionally save to a text file.
    
    Parameters
    ----------
    model : CPLEX model
        The optimization model to inspect.
    output_file : str, optional
        Path to a text file where all constraints will be saved.
    return_df : bool, default False
        If True, returns a DataFrame containing all constraints info.
    
    Returns
    -------
    pd.DataFrame or None
        DataFrame with columns: ['name', 'expression', 'sense', 'rhs', 'variables', 'coefficients'] if return_df=True.
    """
    import pandas as pd

    constraint_data = []

    for i in range(model.linear_constraints.get_num()):
        row = model.linear_constraints.get_rows(i)
        indices = row.ind
        coefs = row.val
        name = model.linear_constraints.get_names(i) or f"constraint_{i}"
        rhs = model.linear_constraints.get_rhs(i)
        sense = model.linear_constraints.get_senses(i)

        vars_in_constraint = [model.variables.get_names(idx) for idx in indices]
        terms = [f"{coef}*{var}" for coef, var in zip(coefs, vars_in_constraint)]
        expression = " + ".join(terms)

        # Store info for DataFrame or file
        constraint_data.append({
            "name": name,
            "expression": expression,
            "sense": sense,
            "rhs": rhs,
            "variables": vars_in_constraint,
            "coefficients": coefs
        })

    # Save to file if requested
    if output_file:
        with open(output_file, "w") as f:
            for row in constraint_data:
                f.write(f"{row['name']}: {row['expression']} {row['sense']} {row['rhs']}\n")
        print(f"All constraints saved to {output_file}")

    if return_df:
        return pd.DataFrame(constraint_data)


### 💾 save the model

## Solve

In [40]:
def print_solution_summary(model, top_n=20):
    """
    Prints a summarized view of the CPLEX solution for large models.

    Parameters
    ----------
    model : CPLEX model
        Solved CPLEX model.
    top_n : int, default 20
        Number of largest non-zero variables to display.
    save_csv : str or None
        If provided, saves the full solution to a CSV file.
    """
    solution = model.solution
    if not solution.is_primal_feasible():
        print("❌ No feasible solution found.")
        return

    # Objective
    obj_val = solution.get_objective_value()
    print(f"✅ Objective value: {obj_val}\n")
    print(f"Optimal values: {model.solution.get_values()}\n")

    # Get all variables and values
    var_names = model.variables.get_names()
    var_values = solution.get_values()

    # Create DataFrame for easy filtering/sorting
    df_sol = pd.DataFrame({
        "variable": var_names,
        "value": var_values
    })

    # Non-zero variables
    non_zero = df_sol[df_sol["value"] != 0].copy()
    print(f"Total variables: {len(var_names)}")
    print(f"Non-zero variables: {len(non_zero)}\n")

    # Print top N largest values
    print(f"Top {top_n} largest non-zero variables:")
    print(non_zero.sort_values(by="value", ascending=False).head(top_n).to_string(index=False))
    
    return df_sol, solution

## 💾 save the results

## Post-Processing

In [41]:
def extract_solution_df(model):
    """
    Extracts the CPLEX solution into a tidy DataFrame with columns:
    variable, value, period, action, type, edge.

    Parameters
    ----------
    model : CPLEX model
        Solved CPLEX model.

    Returns
    -------
    pd.DataFrame
        DataFrame with solution variables and metadata.
    """
    # Get solution object
    solution = model.solution

    # Get variable names and values
    variable_names = model.variables.get_names()
    variable_values = solution.get_values()

    # Create initial DataFrame
    df = pd.DataFrame({
        'variable': variable_names,
        'value': variable_values
    })

    # --- Extract period safely ---
    df['period'] = df['variable'].str.extract(r'_t(\d+)$')[0]
    df['period'] = pd.to_numeric(df['period'], errors='coerce')  # NaN if no match

    # --- Determine action ---
    def determine_action(var):
        if var.startswith('C'):
            return 'construct'
        elif var.startswith('M'):
            return 'maintain'
        elif var.startswith('U'):
            return 'upgrade'
        elif var.startswith('flow'):
            return 'flow'
        return 'Unknown'
    df['action'] = df['variable'].apply(determine_action)

    # --- Extract road type (timber or wide) ---
    df['type'] = df['variable'].str.extract(r'_(timber|wide)_')[0]

    # --- Extract edge (like node8node14) ---
    df['edge'] = df['variable'].str.extract(r'_(node\d+node\d+)')[0]

    return df

In [42]:
def remap_solution_to_coords(solution_df, node_ID_mapping):
    """
    Remaps edges in the solution DataFrame to node coordinates.

    Parameters
    ----------
    solution_df : pd.DataFrame
        DataFrame containing at least the 'edge' column.
    node_ID_mapping : dict
        Maps node IDs (e.g., 'node8') to coordinates (tuple or list, e.g., (lat, lon)).

    Returns
    -------
    pd.DataFrame
        Updated DataFrame with coordinate columns:
        node1, node2, node1_coord, node2_coord, node1_lat, node1_lon, node2_lat, node2_lon
    """
    # Ensure we work on a copy to avoid SettingWithCopyWarning
    df = solution_df.copy()

    # Split edge into nodes
    df[['node1', 'node2']] = df['edge'].str.extract(r'(node\d+)(node\d+)')

    # Map full node IDs to coordinates
    df['node1_coord'] = df['node1'].map(node_ID_mapping)
    df['node2_coord'] = df['node2'].map(node_ID_mapping)

    # Split coordinates into lat/lon
    df[['node1_lat','node1_lon']] = pd.DataFrame(df['node1_coord'].tolist(), index=df.index)
    df[['node2_lat','node2_lon']] = pd.DataFrame(df['node2_coord'].tolist(), index=df.index)

    return df

## All-in-1 verification workflow

In [43]:
# Helper Functions

def prepare_verify_df(solution_df, edge_attr_df):
    """
    Merge solution with edge attributes and compute cost for each row.
    """
    verify_df = solution_df.merge(edge_attr_df, left_on="edge", right_on="EdgeID", how='inner')
    
    # Define conditions for cost assignment
    conditions = [
        (verify_df["action"] == "construct") & (verify_df["type"] == "timber"),
        (verify_df["action"] == "construct") & (verify_df["type"] == "wide"),
        (verify_df["action"] == "maintain") & (verify_df["type"] == "timber"),
        (verify_df["action"] == "maintain") & (verify_df["type"] == "wide"),
        (verify_df["action"] == "upgrade"),
    ]
    
    choices = [
        verify_df["Build5m"],
        verify_df["Build10m"],
        verify_df["Maintain5m"],
        verify_df["Maintain10m"],
        verify_df["Upgrade"],
    ]
    
    verify_df["cost"] = np.select(conditions, choices, default=0)
    verify_df.drop(columns=["Build5m", "Build10m", "Maintain5m", "Maintain10m", "Upgrade"], inplace=True)
    
    return verify_df

def check_solution_value(verify_df, solution):
    """
    Verify that the calculated cost matches the solver objective.
    """
    calc_cost = verify_df.loc[verify_df.value != 0, "cost"].sum()
    sol_cost = solution.get_objective_value()
    
    print("🔎 Consistency Check")
    print("-------------------")
    print(f"Calculated cost (from verify_df): {calc_cost}")
    print(f"Solver-reported objective value : {sol_cost}")
    
    if abs(calc_cost - sol_cost) < 1e-6:
        print("✅ Consistent: values match within tolerance.")
    else:
        print("⚠️ Inconsistency detected:")
        print(f"Difference: {calc_cost - sol_cost}")

def check_initialization(verify_df):
    """
    Verify no edges are active at period 0.
    """
    init_edges = verify_df[(verify_df["value"] != 0) & (verify_df["period"] == 0)]
    print("----------------------------------------")
    print("🔎 Initialization Check (period 0)")
    if init_edges.empty:
        print("✅ No edges active at period 0 (clean initialization).")
    else:
        print("⚠️ Some edges are already active at initialization:")
        print(init_edges[["node_min", "node_max", "value", "cost"]])

def check_maintenance_rules(verify_df, road_type, allowed_actions):
    """
    Verify that maintenance/upgrades are only applied after valid previous actions.
    """
    df = verify_df[verify_df["type"] == road_type].copy()
    check_df = df[
        (df["action"].isin(["maintain", "upgrade"])) &
        (df["value"] == 1) &
        (df["period"] >= 1)
    ].copy()
    
    reference_df = df[
        (df["action"].isin(allowed_actions)) &
        (df["value"] == 1)
    ][["edge", "period"]]
    
    check_df["check_period"] = check_df["period"] - 1
    validated_df = check_df.merge(
        reference_df,
        left_on=["edge", "check_period"],
        right_on=["edge", "period"],
        how="left",
        indicator=True
    )
    
    violations = validated_df[validated_df["_merge"] == "left_only"]
    print("----------------------------------")
    print(f"🔎 {road_type.capitalize()} Road Maintenance/Upgrade Check")
    if violations.empty:
        print(f"✅ All good — every {road_type} road maintain/upgrade has valid prior action.")
    else:
        print(f"⚠️ {len(violations)} violation(s) found.")
        print(violations[["edge", "period_x", "action"]])

def check_flow_support(verify_df, swap_direction):
    """
    Verify that flow only occurs on edges with a supporting construction/maintenance/upgrade action.
    
    Timber flows can use timber or wide support.
    Wide flows must use wide support.
    
    Parameters
    ----------
    verify_df : pd.DataFrame
        Must contain columns: ['action','edge','period','type','value']
    swap_direction : function
        Function to reverse the edge string (for checking reverse flows)
    """
    # Step 1: Get all Flow actions with non-zero values
    flow_df = verify_df[
        (verify_df["action"] == "flow") & (verify_df["value"] != 0)
    ][["edge", "period", "type"]].drop_duplicates()
    
    # Step 2: Get all supporting actions (construct/maintain)
    support_df = verify_df[
        (verify_df["action"].isin(["construct", "maintain"])) & (verify_df["value"] == 1)
    ][["edge", "period", "type"]].drop_duplicates()
    
    # Step 3: Get upgrade actions (expand to both types)
    upgrade_df = verify_df[
        (verify_df["action"] == "upgrade") & (verify_df["value"] == 1)
    ][["edge","period"]].drop_duplicates()
    
    # Expand upgrades to both types for timber flows
    upgrade_expanded = upgrade_df.assign(key=1).merge(
        pd.DataFrame({"type": ["timber","wide"], "key":1}),
        on="key"
    ).drop("key", axis=1)
    
    # Combine support and upgrades
    combined_support_df = pd.concat([support_df, upgrade_expanded], ignore_index=True)
    
    # Step 4: Validate flows
    def is_supported(row):
        if row["type"] == "timber":
            # Timber flow can use timber or wide support
            return ((combined_support_df.edge == row.edge) &
                    (combined_support_df.period == row.period) &
                    (combined_support_df.type.isin(["timber","wide"]))).any()
        else:
            # Wide flow must use wide support
            return ((combined_support_df.edge == row.edge) &
                    (combined_support_df.period == row.period) &
                    (combined_support_df.type == "wide")).any()
    
    flow_df["supported"] = flow_df.apply(is_supported, axis=1)
    
    # Step 5: Check reversed direction for unsupported flows
    reverse_flow_df = flow_df[~flow_df["supported"]].copy()
    reverse_flow_df["edge"] = reverse_flow_df["edge"].apply(swap_direction)
    reverse_flow_df["supported"] = reverse_flow_df.apply(is_supported, axis=1)
    
    missing_flows = reverse_flow_df[~reverse_flow_df["supported"]]
    
    # Report
    print("----------------------------------------")
    print("🔎 Flow Support Check")
    if missing_flows.empty:
        print("✅ All flows have valid support according to asymmetric rules.")
    else:
        print(f"⚠️ {len(missing_flows)} flow(s) are missing valid support even after checking reversed direction:")
        print(missing_flows[["edge","period","type"]])
    
    return missing_flows

def check_flow_balance(verify_df, V_source, V_exit):
    """
    Check flow conservation for all nodes except sources and exits.
    """
    source_nodes = set(V_source.values()) if isinstance(V_source, dict) else set(V_source)
    exit_nodes = set(V_exit.values()) if isinstance(V_exit, dict) else set(V_exit)
    flow_df = verify_df[(verify_df["action"] == "flow") & (verify_df["value"] != 0)]
    total_violations = 0

    for road_type, type_group in flow_df.groupby("type"):
        for period, df_group in type_group.groupby("period"):
            outgoing = df_group.groupby("node1_ID")["value"].sum().rename("outgoing")
            incoming = df_group.groupby("node2_ID")["value"].sum().rename("incoming")
            flow_balance = pd.concat([outgoing, incoming], axis=1).fillna(0)
            flow_balance["diff"] = flow_balance["incoming"] - flow_balance["outgoing"]
            
            unbalanced = flow_balance[
                (flow_balance["diff"] != 0) &
                (~flow_balance.index.isin(source_nodes)) &
                (~flow_balance.index.isin(exit_nodes))
            ]
            if not unbalanced.empty:
                print(f"⚠️ Found {len(unbalanced)} unbalanced node(s):")
                total_violations += len(unbalanced)
                for node, row in unbalanced.iterrows():
                    print(f"  Node {node}: in={row['incoming']}, out={row['outgoing']}, diff={row['diff']}")
    print("----------------------------------------")
    print(" 🔎 Flow Balance Report ===")
    if total_violations == 0:
        print("✅ All nodes satisfy flow conservation (except sources/exits).")
    else:
        print(f"⚠️ Total {total_violations} unbalanced node-period(s) detected.")

def check_source_outflow(verify_df, needroad_w_s_t, V_source_dict):
    """
    Check that outflow from source nodes matches expected.
    """
    node_to_s = {v: k for k, v in V_source_dict.items()}
    flow_out_df = verify_df[
        (verify_df["action"] == "flow") &
        (verify_df["node1"].isin(node_to_s.keys()))
    ].copy()
    flow_out_df["s"] = flow_out_df["node1"].map(node_to_s)

    actual_outflow = (
        flow_out_df.groupby(["type", "s", "period"])["value"]
        .sum()
        .rename("actual")
        .reset_index()
    )
    actual_outflow["expected"] = actual_outflow.apply(
        lambda row: needroad_w_s_t.get((row["type"], row["s"], row["period"]), 0),
        axis=1
    )
    actual_outflow["diff"] = actual_outflow["actual"] - actual_outflow["expected"]

    mismatches = actual_outflow[actual_outflow["diff"] != 0]
    print("----------------------------------------")
    print("=== Source Node Outflow Check ===")
    if mismatches.empty:
        print("✅ All sources have correct outflow as expected.")
    else:
        print(f"⚠️ Found {len(mismatches)} mismatch(es) in source outflow:")
        print(mismatches[["type","s","period","actual","expected","diff"]])

def check_double_construction(verify_df):
    """
    Check for edges that are constructed as both wide and timber in the same period.
    """
    nonimaginary_edges_df = verify_df[
        (verify_df["value"] ==1) &
        (verify_df.action.isin(['construct', 'maintain','upgrade'])) &
        (~verify_df.has_source)
    ].copy()
    
    nonimaginary_edges_df["node_min"] = nonimaginary_edges_df[["node1", "node2"]].min(axis=1)
    nonimaginary_edges_df["node_max"] = nonimaginary_edges_df[["node1", "node2"]].max(axis=1)
    
    grouped = nonimaginary_edges_df.groupby(["node_min", "node_max", "period"])["type"].apply(set).reset_index()
    double_construction = grouped[grouped["type"].apply(lambda x: {"wide", "timber"}.issubset(x))]
    print("----------------------------------------")  
    print("=== Double Construction Check ===")
    if double_construction.empty:
        print("✅ No edges are constructed as both 'wide' and 'timber' in the same period.")
    else:
        print(f"⚠️ Found {len(double_construction)} edge(s) with double construction:")
        print(double_construction)

# -----------------------
# Full Workflow Function
# -----------------------

def verify_solution_full(solution_df, edge_attr_df, solution, V_source_dict, V_exit, needroad_w_s_t, swap_direction):
    """
    Run all verification checks on the solution.
    """
    print("\n=== Preparing Verification DataFrame ===")
    verify_df = prepare_verify_df(solution_df, edge_attr_df)

    check_solution_value(verify_df, solution)
    check_initialization(verify_df)
    check_maintenance_rules(verify_df, "timber", ["construct", "maintain"])
    check_maintenance_rules(verify_df, "wide", ["construct", "maintain", "upgrade"])
    check_flow_support(verify_df, swap_direction)
    check_flow_balance(verify_df, V_source_dict, V_exit_nIDs)
    check_source_outflow(verify_df, needroad_w_s_t, V_source_dict)
    check_double_construction(verify_df)
    
    print("Verification workflow completed.")
    return verify_df


## Full Workflow

In [44]:
os.makedirs("2_Model", exist_ok=True)
model_dir="2_Model_Solution"

harv_sched_file_paths = {
    "mres": r'1_Preprocessed_Data\2_Stands_Access_Requirements\mres_stands_access_needs.csv',
    "mwood": r'1_Preprocessed_Data\2_Stands_Access_Requirements\mwood_stands_access_needs.csv',
    "stake": r'1_Preprocessed_Data\2_Stands_Access_Requirements\stake_stands_access_needs.csv'
}
# basedir to get the components
base_dir = r"1_Preprocessed_Data\4_Road_Network_Graphs"

# define fixed sets
T, W = define_fixed_sets()

### loop

In [ ]:
# Loop over harvesting schedules ---
for hsched_name, path in harv_sched_file_paths.items():
    print(f"\n--- Harvesting schedule: {hsched_name} ---")
    # Loop over components and run workflow ---
    for folder_name in os.listdir(base_dir):
        in_component_dir = os.path.join(base_dir, folder_name)
        if not os.path.isdir(in_component_dir):
            continue  # Skip non-directories

        out_dir = os.path.join(model_dir, hsched_name, folder_name)
        os.makedirs(out_dir, exist_ok=True)

        print(f"\n=== Processing Component: {folder_name} ===")

        # --- Load nodes ---
        V_source, V_exit, V, nodes_df = load_nodes(in_component_dir)

        # --- Map node IDs ---
        node_ID_mapping, reversed_node_ID_mapping, V_nodeIDs, V_source_nIDs, V_exit_nIDs = build_node_mappings(V, V_source, V_exit)

        # --- Transit nodes ---
        V_transit = get_transit_nodes(V_nodeIDs, V_source_nIDs, V_exit_nIDs)

        # --- Load arcs & attributes ---
        A, arcs_fw, arcs_bw, edge_attr_df = load_and_verify_arcs(in_component_dir)

        # --- Map arcs to node IDs ---
        A, A_fw, A_bw = replace_coord_with_IDs(A, arcs_fw, arcs_bw)
        
        # --- Process edges into tuples/strings ---
        A, A_fw, A_bw, E = process_edge_pairs(A, A_fw, A_bw)

        # create fw bw mappings
        fw_to_bw_map, bw_to_fw_map= create_forward_backward_mapping(A_fw, A_bw, swap_direction)
        
        # --- Build source dictionary ---
        V_source_dict, S = build_source_dict(V_source, reversed_node_ID_mapping)

        # --- Build arcs ---
        source_arcs_ingoing, source_arcs_outgoing = get_source_arcs(A, V_source_dict)
        exit_arcs_ingoing, totalin, exit_arcs_outgoing, totalout = build_exit_arcs(V_exit_nIDs, A)
        ingoing_per_transitnode, outgoing_per_transitnode, total_in, total_out = build_transitnode_arcs(V_transit, A)

        # --- Load parameters & costs & accessneeds---
        reverse_mapping = {f'({x[0]}, {x[1]})': short_name for short_name, x in node_ID_mapping.items()}
        edge_attr_df = load_and_process_edge_attributes(f'{in_component_dir}/arcs_with_attributes.csv', reverse_mapping)
        CostC, CostM, CostU = build_cost_dicts(E, W, edge_attr_df)
        accessneeds_df, wide_accessneeds_df = load_access_needs(path, filter_stands=S)
        accessneeds_df_dict = {'timber': accessneeds_df, 'wide': wide_accessneeds_df}

        # --- Needroad and maxflow ---
        needroad_w_s_t = set_needroad_parameters(S, T, accessneeds_df_dict)
        maxflow_results = set_maxflow_params(S, T, needroad_w_s_t, ['timber', 'wide'])
        maxflow_w_t = {**maxflow_results['timber'], **maxflow_results['wide']}

        # --- Decision variables ---
        C, M, U = create_binary_decision_variables(E, W, T)
        flow = create_integer_flow_variables(A, W, T)

        # --- Build CPLEX model ---
        model = Cplex()
        model.set_problem_type(Cplex.problem_type.LP)
        model.variables.delete()
        model.linear_constraints.delete()

        # --- Add variables and objective ---
        objective_terms, objective_coeffs = add_binary_variables_to_model(model, C, M, U, CostC, CostM, CostU)
        n_action_variables = verify_binary_variables(model, E, T, objective_coeffs, 5)
        set_objective(model, objective_terms, objective_coeffs)
        add_flow_variables(model, flow, lb=0, ub=9)
        n_flow_variables = verify_flow_variables(model, n_action_variables, A, W, T)
        arc_check_results = verify_transitnode_arcs(flow, ingoing_per_transitnode, outgoing_per_transitnode)

        # --- save Variables
        save_binary_variables_info(model, CostC, CostM, CostU, f'{out_dir}')
        save_flow_variables_info(model, n_flow_variables, f'{out_dir}')

        # --- Add constraints ---
        # 1. Wide road flow enforcing road existence
        verification_wide = add_wide_flow_constraints(model, flow, C, M, U, maxflow_w_t, fw_to_bw_map, T, A_fw)
        expected_constraints_group1 = 5 * len(E)

        # 2. Timber road flow enforcing road existence
        verification_timber = add_timber_flow_constraints(model, flow, C, M, U, maxflow_w_t, fw_to_bw_map, T, E)
        expected_constraints_group2 = 5 * len(E)  # adjust if multiplicity of T is relevant

        # 3. Flow conservation at transit nodes (all road types)
        verification_flow_conservation = add_flow_conservation_constraints(model, flow, ingoing_per_transitnode, outgoing_per_transitnode, W, T)
        expected_constraints_group3 = 2 * len(V_transit) * len(T)

        # 4. Outflow only if stand needs road (all road types)
        verification_outflow = add_outflow_needroad_constraints(model, flow, needroad_w_s_t, source_arcs_outgoing, W, T)
        expected_constraints_group4 = len(V_source_dict) * len(T) * 2  # adjust if number of periods differs

        # 5. No inflow to source nodes
        verification_no_inflow = add_no_inflow_source_constraints(model, flow, source_arcs_ingoing, T)
        expected_constraints_group5 = len(V_source_dict) * len(T)  # adjust if number of periods differs

        # 6. Maintenance/Upgrade timber roads
        verification_maintain_timber = add_maintain_upgrade_constraints(model, C, M, U, E, T, road_type='timber')
        expected_constraints_group6 = len(T) * len(E)  # adjust if number of periods differs

        # 7. Maintenance/Upgrade wide roads
        verification_maintain_wide = add_maintain_upgrade_wide_constraints(model, C, M, U, E, T, road_type='wide')
        expected_constraints_group7 = len(T) * len(E)  # adjust if number of periods differs

        # --- Verify and store constraints ---
        verify_and_store_constraints(model, verification_wide, "Constraint_flow_enforcing_road", 5*len(E), f"{out_dir}/Constraints_wideflow_enforce_wideroad.txt")
        verify_and_store_constraints(model, verification_timber, "Constraint_timberflow_enforcing_road", 5*len(E), f"{out_dir}/Constraints_timberflow_enforce_road.txt")
        verify_and_store_constraints(model, verification_flow_conservation, "Constraint_flow_conservation", 2*len(V_transit)*len(T), f"{out_dir}/Constraints_flow_conservation.txt")
        verify_and_store_constraints(model, verification_outflow, "Constraint_outflow_iff_needroad", len(V_source_dict)*len(T)*2, f"{out_dir}/Constraints_outflow_iff_needroad.txt")
        verify_and_store_constraints(model, verification_no_inflow, "Constraint_no_inflow_source", len(V_source_dict)*len(T), f"{out_dir}/Constraints_no_inflow_to_source.txt")
        verify_and_store_constraints(model, verification_maintain_timber, "Constraint_maintain_upgrade", len(T)*len(E), f"{out_dir}/Constraints_maintain_upgrade_timber.txt")
        verify_and_store_constraints(model, verification_maintain_wide, "Constraint_maintain_upgrade_wide", len(T)*len(E), f"{out_dir}/Constraints_maintain_upgrade_wide.txt")

        # --- Save model files ---
        model.write(os.path.join(out_dir, "model.lp"))
        model.write(os.path.join(out_dir, "model.sav"))
        model.write(os.path.join(out_dir, "model.mps"))

        # --- Solve model ---
        model.solve()

        # --- Save solution ---
        df_sol, solution = print_solution_summary(model, top_n=5)
        df_sol.to_csv(os.path.join(out_dir, "solution_full.csv"), index=False)
        df_sol[df_sol['value'] != 0].to_csv(os.path.join(out_dir, "solution_nonzero.csv"), index=False)
        with open(os.path.join(out_dir, "solution_summary.txt"), "w") as f:
            f.write(f"Objective value: {model.solution.get_objective_value()}\n")
            f.write(f"Total variables: {len(model.variables.get_names())}\n")
            f.write(f"Non-zero variables: {len(df_sol[df_sol['value'] != 0])}\n")

        # --- Post Processing ---
        full_solution_df = extract_solution_df(model)
        solution_df = full_solution_df[full_solution_df.value!=0]
        solution_df = remap_solution_to_coords(solution_df, node_ID_mapping)
        full_solution_df = remap_solution_to_coords(full_solution_df, node_ID_mapping)

        #--- Verification Workflow ---

        # --- store component solution for postprocessing to merge and visualize and use it

        print(f"=== Finished Component: {folder_name} ===")
    # --- merge the component solutions into one big solution

    # --- visualize  
    # f  
    print(f'--- FINISHED {hsched_name} ---') 


--- Harvesting schedule: mres ---

=== Processing Component: comp_1 ===
Total nodes: 43
Transit nodes: 13
Source nodes: 7
Exit nodes: 23
Total arcs: 196, Forward arcs: 98, Backward arcs: 98
✅ Forward arcs are the reverse of backward arcs.
55 ingoing to source nodes in total
55 outgoing from source nodes in total
Decision variables for C, M, U created.
Decision variables for C, M, U added to model.
2940 action variables
5*len(E)*(len(T)+1) = 2940
2940 objective coefficients
✅ Verification passed: variable count match expected
✅ Verification passed: coefficient count match expected
Linear objective function assigned and set to minimize.
1960 flow variables added to the model.
4900 total variables
1960 flow variables
Expected flow variables (len(A)*len(T)*len(W)) = 1960
✅ Flow variable count matches expected

=== Transit Node Arc Verification ===
Missing arcs in ingoing per transit node: 0
Missing arcs in outgoing per transit node: 0
✅ All transit node arcs are properly represented in th

Default objective names obj1, obj2 ... being created.


Done.
Done.
Done.
Done for 'timber'.
Done for wide.

=== Constraint Verification: Constraint_flow_enforcing_road ===
Total constraints in model: 2195
Constraints added in this group: 490
Expected number of constraints for this group: 490
✅ Check passed: 490 constraints added as expected.
Constraints written to: 2_Model_Solution\mres\comp_1/Constraints_wideflow_enforce_wideroad.txt

=== Constraint Verification: Constraint_timberflow_enforcing_road ===
Total constraints in model: 2195
Constraints added in this group: 490
Expected number of constraints for this group: 490
✅ Check passed: 490 constraints added as expected.
Constraints written to: 2_Model_Solution\mres\comp_1/Constraints_timberflow_enforce_road.txt

=== Constraint Verification: Constraint_flow_conservation ===
Total constraints in model: 2195
Constraints added in this group: 130
Expected number of constraints for this group: 130
✅ Check passed: 130 constraints added as expected.
Constraints written to: 2_Model_Solution\mres

Default objective names obj1, obj2 ... being created.


Done for wide.

=== Constraint Verification: Constraint_flow_enforcing_road ===
Total constraints in model: 6545
Constraints added in this group: 1435
Expected number of constraints for this group: 1435
✅ Check passed: 1435 constraints added as expected.
Constraints written to: 2_Model_Solution\mres\comp_10/Constraints_wideflow_enforce_wideroad.txt

=== Constraint Verification: Constraint_timberflow_enforcing_road ===
Total constraints in model: 6545
Constraints added in this group: 1435
Expected number of constraints for this group: 1435
✅ Check passed: 1435 constraints added as expected.
Constraints written to: 2_Model_Solution\mres\comp_10/Constraints_timberflow_enforce_road.txt

=== Constraint Verification: Constraint_flow_conservation ===
Total constraints in model: 6545
Constraints added in this group: 430
Expected number of constraints for this group: 430
✅ Check passed: 430 constraints added as expected.
Constraints written to: 2_Model_Solution\mres\comp_10/Constraints_flow_con

Default objective names obj1, obj2 ... being created.


Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve eliminated 15861 rows and 36226 columns.
MIP Presolve added 658 rows and 0 columns.
MIP Presolve modified 7896 coefficients.
Aggregator did 106 substitutions.
Reduced MIP has 8166 rows, 14618 columns, and 40152 nonzeros.
Reduced MIP has 8038 binaries, 6580 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.16 sec. (56.83 ticks)
Found incumbent of value 7740845.000000 after 0.28 sec. (116.00 ticks)
Probing time = 0.02 sec. (1.25 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 1681 rows and 3033 columns.
Reduced MIP has 6485 rows, 11585 columns, and 32174 nonzeros.
Reduced MIP has 6445 binaries, 5140 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.05 sec. (16.14 ticks)
Probing time = 0.01 sec. (1.01 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 6485 rows, 11585 columns, and 32174 nonzeros.
Reduced MIP ha

## 👁 Visualize the result

#### prep: load shapefile

### plot shapefile

### plot the real edges

### plot the solution